In [1]:
import pandas as pd, numpy as np
from shapely.geometry import Polygon
from shapely.geometry import Point
import geopandas
import matplotlib.pyplot as plot

In [6]:
path = 'Output/' # select folder with data in it

#specify datatypes so FIPS numbers, etc. read in properly
dtypes = {'Geo_STATE': 'str', 'Geo_FIPS': 'str', 'Geo_COUNTY': 'str', 'h_tract': 'str', 'FIPS_County': 'str'}

tract_data = pd.read_csv(path + 'tract_data.csv', delimiter=',', dtype=dtypes)

tract_data.head(1)

,Unnamed: 0,Geo_FIPS,alljobs,ltbajobs,lt40kjobs,county_labor,county_unemp,county_med_rent,county_med_value,CBSA Code,...,bach_degree,pop_over_25,hh_kids,total_hh,med_rent,med_housevalue,unemp_civ,civ_labor_force,area_sqmi,HU_density
0,0,06001400100,763651.0,358749.0,301972.0,862353,61327,1432,593500,41860.0,...,1981.0,2478.0,241.0,1292.0,3202.0,1074100.0,75,1643,2.657,486.262695


In [7]:
#specify datatypes so FIPS numbers, etc. read in properly
dtypes2 = {'GEOID': 'str', 'Agency': 'str', 'Stop ID': 'str'}

station_data = pd.read_csv('best_matches.csv', delimiter=',', dtype=dtypes2)

station_small = station_data[['GEOID','Agency','Stop ID','station_area','tract_area','geometry','overlap_area','key','mode']]
station_small.head(1)

,GEOID,Agency,Stop ID,station_area,tract_area,geometry,overlap_area,key,mode
0,06043000400,Yosemite Valley Shuttle System,766882,0.507711,1025.605497,"POLYGON ((801245.0431922717 4182865.720460022,...",0.507711,Yosemite Valley Shuttle System766882,bus


In [8]:
#merge together datasets by census tract
merge_data = pd.merge(station_small, tract_data, left_on="GEOID", right_on="Geo_FIPS", how="left")
merge_data.head(2)

,GEOID,Agency,Stop ID,station_area,tract_area,geometry,overlap_area,key,mode,Unnamed: 0,...,bach_degree,pop_over_25,hh_kids,total_hh,med_rent,med_housevalue,unemp_civ,civ_labor_force,area_sqmi,HU_density
0,06043000400,Yosemite Valley Shuttle System,766882,0.507711,1025.605497,"POLYGON ((801245.0431922717 4182865.720460022,...",0.507711,Yosemite Valley Shuttle System766882,bus,3601,...,568.0,1579.0,175.0,520.0,533.0,363100.0,108,1466,394.127991,1.319368
1,06071009910,Victor Valley Transit Authority,813390,0.507711,4.403783,"POLYGON ((1019267.173736518 3829647.657939956,...",0.440581,Victor Valley Transit Authority813390,bus,5581,...,283.0,3153.0,637.0,1378.0,1264.0,142400.0,365,2332,1.693000,813.939758


In [11]:
#May want to switch this to a weighted average from a simple average
#wm = lambda x: np.average(x, weights=merge_data.loc[x.index, "total_pop"])

avg_cols = ['alljobs','county_med_rent','med_rent']

stop_avgs=merge_data[avg_cols].groupby(merge_data['key']).mean()

stop_avgs.head()

,alljobs,county_med_rent,med_rent
key,,,
AC Transit1000340,1065622.0,1432,770.0
AC Transit1000350,1065622.0,1432,770.0
AC Transit1000370,1065622.0,1432,770.0
AC Transit1000380,1065622.0,1432,770.0
AC Transit1000410,1075287.0,1432,855.0


In [12]:
sum_cols = ['renters', 'occ_HU', 'hispanic', 'black',
       'white', 'asian', 'nhpi', 'total_pop', 'below200pov',
       'total_pov_status', 'rent_occ_HU', 'low_inc_rent_burden',
       'white_pov_tot', 'white_pov', 'black_pov_tot', 'black_pov',
       'asian_pov_tot', 'asian_pov', 'nhpi_pov_tot', 'nhpi_pov',
       'hispanic_pov_tot', 'hispanic_pov', 'sfdetach', 'smallmf', 'medmf',
       'bigmf', 'total_hu', 'vacant_hu', 'total_hu2', 'since2000',
       'before1950', 'total_structure', 'bach_degree', 'pop_over_25',
       'hh_kids', 'total_hh','unemp_civ','civ_labor_force','area_sqmi']

stop_tots=merge_data[sum_cols].groupby(merge_data['key']).sum()

stop_stats = pd.DataFrame(stop_tots.iloc[:,7])

stop_stats['rent_comp'] = stop_avgs['med_rent'] / stop_avgs['county_med_rent']
stop_stats['jobs'] = stop_avgs['alljobs']

stop_stats['pct_rent']=stop_tots['renters'] / stop_tots['occ_HU']

stop_stats['pct_white']=stop_tots['white'] / stop_tots['total_pop']
stop_stats['pct_hispanic']=stop_tots['hispanic'] / stop_tots['total_pop']
stop_stats['pct_black']=stop_tots['black'] / stop_tots['total_pop']
stop_stats['pct_asian']=stop_tots['asian'] / stop_tots['total_pop']

stop_stats['pct_below200pov']=stop_tots['below200pov'] / stop_tots['total_pov_status']

stop_stats['hispanic_pov']=stop_tots['hispanic_pov'] / stop_tots['hispanic_pov_tot']
stop_stats['black_pov']=stop_tots['black_pov'] / stop_tots['black_pov_tot']
stop_stats['asian_pov']=stop_tots['asian_pov'] / stop_tots['asian_pov_tot']
stop_stats['white_pov']=stop_tots['white_pov'] / stop_tots['white_pov_tot']

stop_stats['pct_sfdetach']=stop_tots['sfdetach'] / stop_tots['total_hu']
stop_stats['pct_smallmf']=stop_tots['smallmf'] / stop_tots['total_hu']
stop_stats['pct_bigmf']=stop_tots['bigmf'] / stop_tots['total_hu']
stop_stats['pct_medmf']=stop_tots['medmf'] / stop_tots['total_hu']
stop_stats['pct_vacant']=stop_tots['vacant_hu'] / stop_tots['total_hu2']

stop_stats['pct_since2000']=stop_tots['since2000'] / stop_tots['total_structure']
stop_stats['pct_before1950']=stop_tots['before1950'] / stop_tots['total_structure']

stop_stats['pct_bach_degree']=stop_tots['bach_degree'] / stop_tots['pop_over_25']
stop_stats['pct_hh_kids']=stop_tots['hh_kids'] / stop_tots['total_hh']
stop_stats['pct_unemp_civ']=stop_tots['unemp_civ'] / stop_tots['civ_labor_force']
stop_stats['density']=stop_tots['total_pop'] / stop_tots['area_sqmi']

stop_stats.to_csv(path + 'stop_stats.csv')

stop_stats.head()

,total_pop,rent_comp,jobs,pct_rent,pct_white,pct_hispanic,pct_black,pct_asian,pct_below200pov,hispanic_pov,...,pct_smallmf,pct_bigmf,pct_medmf,pct_vacant,pct_since2000,pct_before1950,pct_bach_degree,pct_hh_kids,pct_unemp_civ,density
key,,,,,,,,,,,,,,,,,,,,,
AC Transit1000340,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,0.308511,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
AC Transit1000350,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,0.308511,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
AC Transit1000370,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,0.308511,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
AC Transit1000380,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,0.308511,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
AC Transit1000410,2993.0,0.597067,1075287.0,0.782577,0.218844,0.08854,0.210157,0.446709,0.546131,0.113636,...,0.145415,0.606734,0.171920,0.021490,0.341691,0.270774,0.350773,0.116398,0.096386,11645.914093


In [16]:
matpath = 'Matlab/'
stop_stats.iloc[:,1:23].to_csv(matpath + 'formatlab3.csv')

In [17]:
matpath = 'Matlab/' # select folder with data in it

clusters = pd.read_csv(matpath + 'clusters.csv', header=None, names=['Cluster'], delimiter=',')

print(len(clusters))
clusters.head(10)

10663


,Cluster
0,4
1,4
2,4
3,4
4,4
5,4
6,2
7,4
8,2
9,3


In [18]:
outpath = 'Output/' # select folder with data in it

stats = pd.read_csv(outpath + 'stop_stats.csv', delimiter=',')

print(len(stats))
stats.head()

10663


,key,total_pop,rent_comp,jobs,pct_rent,pct_white,pct_hispanic,pct_black,pct_asian,pct_below200pov,...,pct_smallmf,pct_bigmf,pct_medmf,pct_vacant,pct_since2000,pct_before1950,pct_bach_degree,pct_hh_kids,pct_unemp_civ,density
0,AC Transit1000340,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
1,AC Transit1000350,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
2,AC Transit1000370,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
3,AC Transit1000380,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.056383,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986
4,AC Transit1000410,2993.0,0.597067,1075287.0,0.782577,0.218844,0.08854,0.210157,0.446709,0.546131,...,0.145415,0.606734,0.171920,0.021490,0.341691,0.270774,0.350773,0.116398,0.096386,11645.914093


In [19]:
stats_cluster = pd.concat([stats, clusters], axis=1, join_axes=[stats.index])

stats_cluster.head()

,key,total_pop,rent_comp,jobs,pct_rent,pct_white,pct_hispanic,pct_black,pct_asian,pct_below200pov,...,pct_bigmf,pct_medmf,pct_vacant,pct_since2000,pct_before1950,pct_bach_degree,pct_hh_kids,pct_unemp_civ,density,Cluster
0,AC Transit1000340,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986,4
1,AC Transit1000350,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986,4
2,AC Transit1000370,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986,4
3,AC Transit1000380,4888.0,0.537709,1065622.0,0.764959,0.132160,0.04644,0.090221,0.684534,0.571678,...,0.773258,0.124849,0.064438,0.200967,0.328232,0.288792,0.126991,0.072036,18103.702986,4
4,AC Transit1000410,2993.0,0.597067,1075287.0,0.782577,0.218844,0.08854,0.210157,0.446709,0.546131,...,0.606734,0.171920,0.021490,0.341691,0.270774,0.350773,0.116398,0.096386,11645.914093,4


In [20]:
cols2 = ['total_pop', 'pct_rent', 'pct_white', 'pct_hispanic',
       'pct_black', 'pct_asian', 'pct_below200pov', 'hispanic_pov',
       'black_pov', 'asian_pov', 'white_pov', 'pct_sfdetach', 'pct_smallmf',
       'pct_bigmf', 'pct_medmf', 'pct_vacant', 'pct_since2000',
       'pct_before1950', 'pct_bach_degree', 'pct_hh_kids', 'pct_unemp_civ',
       'density','rent_comp','jobs']

cluster_avgs=stats_cluster[cols2].groupby(stats_cluster['Cluster']).mean()

cluster_avgs.to_csv(outpath + 'cluster_avgs.csv')


In [21]:
stats_cluster['Cluster'].groupby(stats_cluster['Cluster']).count()

Cluster
1    2263
2    1035
3    3275
4    1541
5    2549
Name: Cluster, dtype: int64

In [33]:
print(len(stats_cluster))

10663


In [41]:
stop_path = 'station_id/'

dtypes = {'Stop ID': 'str', 'Name': 'str', 'Agency': 'str', 'OB Routes': 'str', 'IB Routes': 'str'}
stops = pd.read_csv(stop_path + 'AllBus.csv', delimiter=',', dtype=dtypes)

dtypes2 = {'Stop ID': 'str', 'Name': 'str', 'Agency': 'str'}
stations = pd.read_csv(stop_path + 'AllRail.csv', delimiter=',', dtype=dtypes2)
print(len(stops))
print(len(stations))

all_stops = stops[['Agency','Name','Stop ID','X','Y']].append(stations, ignore_index=True)
all_stops['key'] = all_stops['Agency'] + all_stops['Stop ID']

print(len(all_stops))

unique_stops = all_stops.drop_duplicates(subset=['key'], keep=False)

print(len(unique_stops))

unique_stops.head(2)

98913
1445
100358
58908


,Agency,Name,Stop ID,Unnamed: 0,X,Y,key
0,Caltrain,San Jose Caltrain Station,777402,NaN,-121.901985,37.330196,Caltrain777402
1,Caltrain,Tamien Caltrain Station,777403,NaN,-121.883403,37.311638,Caltrain777403


79479


In [43]:
station_cluster = pd.merge(stats_cluster, all_stops, on='key', how="left")
print(len(station_cluster))

17355


In [ ]:
test = station_cluster[(pd.isnull(bus_union['Stop ID'])==False)]

In [ ]:
all_stops['Coordinates'] = list(zip(all_stops.X, all_stops.Y))
all_stops['Coordinates'] = all_stops['Coordinates'].apply(Point)
geo_all = geopandas.GeoDataFrame(all_stops, geometry='Coordinates')

geo_all.crs = {'init': 'epsg:4269'}
print(geo_all.crs)

geo_all = geo_all.to_crs(mtc_crs)
print(geo_all.crs)

geo_all['Coordinates'] = geo_all['Coordinates'].buffer(402.33)